In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [54]:
# kutubxonalarni yuklab olish
import torch
import fastai

# versiyalarini tekshirish
print(torch.__version__)
print(fastai.__version__)

 **Model**

In [55]:
#kutubxonalar
from fastai.vision.all import*
from ipywidgets import widgets

#path
path = Path("../input/pnevmoniya/train")
#fls = get_image_files(path)
#failed = verify_images(path)

#Datablock
chests = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_items = get_image_files,
    splitter = RandomSplitter(valid_pct=0.2, seed=70),
    get_y = parent_label,
    item_tfms = Resize(340)
)

#Dataloaders
dls = chests.dataloaders(path)

#Trainning
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(4)

In [56]:
#cheking
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [5]:
upload = widgets.FileUpload()
upload

In [28]:
img = PILImage.create(upload.data[-1])
pred, pred_id, probs = learn.predict(img)
print(f'Bashorat: {pred}')

In [57]:
path_test = Path('../input/pnevmoniya/test')
images = get_image_files(path_test)

In [58]:
prediction = []
for image in images:
    pred, pred_idx, probs = learn.predict(image)
    name = image.name
    prediction.append([image.name,pred])

In [59]:
predicted = pd.DataFrame(prediction)
predicted[1].value_counts()

In [60]:
dict = {'PNEUMONIA':1,'NORMAL':0}
predicted = predicted.replace(dict)

In [61]:
predicted=predicted.sort_values(0)
predicted

In [62]:
df  = pd.read_csv("../input/pnevmoniya/sample_solution.csv")
df

In [63]:
predicted.index = predicted[0]
predicted

In [64]:
predicted.drop(0, axis=1)

In [65]:
df.labels = predicted[1].values
df

In [66]:
df.to_csv('sub_to_pnev.csv',index=False)